In [1]:
# !pip install -q torch torchvision torchaudio transformers datasets accelerate bitsandbytes langchain sentence-transformers faiss-gpu openpyxl pacmap ragatouille

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# First Part
import os
import json
import re
import pickle
import jsonlines
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Second Part
import torch
from tqdm import tqdm
from typing import Optional, List, Tuple
from langchain.vectorstores import FAISS
from ragatouille import RAGPretrainedModel
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain.docstore.document import Document as LangchainDocument

pd.set_option("display.max_colwidth", None)  # This will be helpful when visualizing retriever outputs

/Users/alessiodesogus/anaconda3/envs/modern_nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
path = '/content/drive/MyDrive/mnlpredators-project/' # CHANGE THIS TO YOUR PATH - [NICOLAS]
file_path_doc_processed = path + 'data/doc_processed_8_keywords.pkl'

### Embeddings Vector - Initialization

In [8]:
# Embedding model
EMBEDDING_MODEL_NAME = "thenlper/gte-small"

/Users/alessiodesogus/anaconda3/envs/modern_nlp/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[May 22, 14:53:08] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/Users/alessiodesogus/anaconda3/envs/modern_nlp/lib/python3.10/site-packages/torch/amp/grad_scaler.py:131: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [9]:
# Load the docs_processed from the file
with open(file_path_doc_processed, 'rb') as file:
    docs_processed = pickle.load(file)

sample_size = 1000  # Change this to your desired sample size
docs_sample = random.sample(docs_processed, sample_size) # Randomly sample docs_processed

In [10]:
# Defin the embedding model with the HuggingFaceEmbeddings class
embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    # model_kwargs={"device": "cpu"},
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
)

/Users/alessiodesogus/anaconda3/envs/modern_nlp/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Creation of the Embedding Vector

In [11]:
KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(docs_sample, embedding_model, distance_strategy=DistanceStrategy.COSINE)  # SAMPLE DATABASE
# KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE) # FULL DATABASE

# Save the database
KNOWLEDGE_VECTOR_DATABASE.save_local("faiss_index_8_keywords_sample") # SAMPLE DATABASE
# KNOWLEDGE_VECTOR_DATABASE.save_local("faiss_index_8_keywords") # FULL DATABASE

In [ ]:
# Load the database
# KNOWLEDGE_VECTOR_DATABASE = FAISS.load_local("faiss_index_8_keywords", embedding_model, allow_dangerous_deserialization=True)